In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale, StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection
from warnings import filterwarnings
filterwarnings('ignore')

C:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
#MODEL

In [26]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League','Division','NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'], axis = 1).astype("float64")
X = pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [27]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [28]:
X_train_scaler = scaler.transform(X_train)
X_test_scaler = scaler.transform(X_test)
#standartlaşmış halleri

In [29]:
mlp_model = MLPRegressor().fit(X_train_scaler,y_train)

In [30]:
mlp_model.n_layers_#üç katmanlı 

3

In [35]:
mlp_model = MLPRegressor(hidden_layer_sizes = (100,20)).fit(X_train_scaler, y_train)

In [36]:
mlp_model.hidden_layer_sizes#gizli katmandaki eleman sayısı

(100, 20)

In [37]:
#TAHMİN

In [38]:
mlp_model.predict(X_test_scaler)[0:5]

array([216.06333447, 361.05229006, 623.04377468, 181.30069332,
       242.98712554])

In [39]:
y_pred = mlp_model.predict(X_test_scaler)

In [40]:
np.sqrt(mean_squared_error(y_pred, y_test))

429.9154154611818

In [41]:
#MODEL TUNING

In [43]:
mlp_params = {'alpha':[0.1,0.01,0.02,0.005],
              'hidden_layer_sizes':[(20,20),(100,50,150),(300,200,150)],
              'activation':['relu','logistic']}

In [44]:
mlp_cv_model = GridSearchCV(mlp_model, mlp_params, cv = 10)

In [45]:
mlp_cv_model.fit(X_train_scaler, y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 20)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150)]})

In [46]:
mlp_cv_model.best_params_

{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50, 150)}

In [47]:
mlp_tuned = MLPRegressor(alpha=0.02, hidden_layer_sizes = (100, 50, 150))

In [48]:
mlp_tuned.fit(X_train_scaler, y_train)

MLPRegressor(alpha=0.02, hidden_layer_sizes=(100, 50, 150))

In [49]:
y_tuned_pred = mlp_tuned.predict(X_test_scaler)

In [50]:
np.sqrt(mean_squared_error(y_test,y_tuned_pred))

359.1595204848892